In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1000)

address = './CSV_FILES/GfK_POS_Tracking-Smart_Audio_Home_Systems_DE_&_UK_python.csv'
data = pd.read_csv(address,low_memory=False)
data.columns = ['COUNTRY','CHANNEL' ,'BRAND','MODEL', 'ID','YEAR','PERIOD','SMART CONNECT','SMART H. ECOSYS',
'OPERATING AI','AIRPLAY','GOOGLECAST/HOME','BLUETOOTH','ETHERNET','WIFI','HEIGHT IN MM','WIDTH IN MM','HIGH-RES AUDIO',
'TYPE OF DOCKING','MULTIROOM','NO.SPEAKERBOXES','OUTPUT CHANNEL','Streaming Connection','Streaming Technology',
'USB CONNECTION','WATTAGE TOTAL','Sales Units','Sales Value EUR','Price EUR','Sales Value USD','Price USD']


In [26]:
data.columns = data.columns.str.replace(' ', '_').str.lower()

In [27]:
Germany = data[data.country=='Germany']
GB = data[data.country=='Great Britain']

In [28]:
data.columns

Index(['country', 'channel', 'brand', 'model', 'id', 'year', 'period',
       'smart_connect', 'smart_h._ecosys', 'operating_ai', 'airplay',
       'googlecast/home', 'bluetooth', 'ethernet', 'wifi', 'height_in_mm',
       'width_in_mm', 'high-res_audio', 'type_of_docking', 'multiroom',
       'no.speakerboxes', 'output_channel', 'streaming_connection',
       'streaming_technology', 'usb_connection', 'wattage_total',
       'sales_units', 'sales_value_eur', 'price_eur', 'sales_value_usd',
       'price_usd'],
      dtype='object')

In [29]:
# Must be modified depending on OS and machine
# Germany.to_csv('C:/School/side_projects/GFK_Hackathon/CSV_FILES/Germany.csv')
# GB.to_csv('C:/School/side_projects/GFK_Hackathon/CSV_FILES/GreatBritain.csv')

In [30]:
temp = data['smart_connect']

In [31]:
data.sort_values('id')

,country,channel,brand,model,id,year,period,smart_connect,smart_h._ecosys,operating_ai,airplay,googlecast/home,bluetooth,ethernet,wifi,height_in_mm,width_in_mm,high-res_audio,type_of_docking,multiroom,no.speakerboxes,output_channel,streaming_connection,streaming_technology,usb_connection,wattage_total,sales_units,sales_value_eur,price_eur,sales_value_usd,price_usd
5188,Germany,Consumer Electronic Stores,NOXON,NOVA S,100293423,2016,16-Dec,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,2,238,119,253,126
4423,Germany,Mass Merchandisers/DIYSs,NOXON,NOVA S,100293423,2017,17-Apr,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,2,190,95,203,101
4201,Germany,Mass Merchandisers/DIYSs,NOXON,NOVA S,100293423,2017,17-May,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,8,760,95,829,104
4002,Germany,Mass Merchandisers/DIYSs,NOXON,NOVA S,100293423,2017,17-Jun,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,10,923,92,1035,104
3855,Germany,Mass Merchandisers/DIYSs,NOXON,NOVA S,100293423,2017,17-Jul,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,3,253,84,287,96
3621,Germany,Mass Merchandisers/DIYSs,NOXON,NOVA S,100293423,2017,17-Aug,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,12,929,77,1097,91
3468,Germany,Mass Merchandisers/DIYSs,NOXON,NOVA S,100293423,2017,17-Sep,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,3,287,96,342,114
3236,Germany,Mass Merchandisers/DIYSs,NOXON,NOVA S,100293423,2017,17-Oct,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,8,690,86,811,101
3027,Germany,Mass Merchandisers/DIYSs,NOXON,NOVA S,100293423,2017,17-Nov,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,9,820,91,955,106
2557,Germany,Mass Merchandisers/DIYSs,NOXON,NOVA S,100293423,2018,18-Jan,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,10,973,97,1174,117


In [32]:
data.sort_values(['id', 'year', 'period'])

,country,channel,brand,model,id,year,period,smart_connect,smart_h._ecosys,operating_ai,airplay,googlecast/home,bluetooth,ethernet,wifi,height_in_mm,width_in_mm,high-res_audio,type_of_docking,multiroom,no.speakerboxes,output_channel,streaming_connection,streaming_technology,usb_connection,wattage_total,sales_units,sales_value_eur,price_eur,sales_value_usd,price_usd
5188,Germany,Consumer Electronic Stores,NOXON,NOVA S,100293423,2016,16-Dec,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,2,238,119,253,126
5148,Germany,Mass Merchandisers/DIYSs,NOXON,NOVA S,100293423,2016,16-Dec,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,10,789,79,839,84
5006,Germany,Consumer Electronic Stores,NOXON,NOVA S,100293423,2017,17-Jan,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,1,139,139,145,145
4953,Germany,Mass Merchandisers/DIYSs,NOXON,NOVA S,100293423,2017,17-Jan,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,12,1666,139,1743,145
4807,Germany,Consumer Electronic Stores,NOXON,NOVA S,100293423,2017,17-Feb,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,1,139,139,150,150
4789,Germany,Mass Merchandisers/DIYSs,NOXON,NOVA S,100293423,2017,17-Feb,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,2,199,100,215,107
4608,Germany,Consumer Electronic Stores,NOXON,NOVA S,100293423,2017,17-Mar,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,1,100,100,105,105
4561,Germany,Mass Merchandisers/DIYSs,NOXON,NOVA S,100293423,2017,17-Mar,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,7,737,105,776,111
4408,Germany,Consumer Electronic Stores,NOXON,NOVA S,100293423,2017,17-Apr,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,4,400,100,426,107
4423,Germany,Mass Merchandisers/DIYSs,NOXON,NOVA S,100293423,2017,17-Apr,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,2,190,95,203,101


In [33]:
dates = data.period.unique()

In [34]:
dates.sort()

AttributeError: 'Categorical' object has no attribute 'sort'

In [ ]:
dates

In [ ]:
months = ['16-Dec',
          '17-Jan', '17-Feb', '17-Mar', '17-Apr', '17-May', '17-Jun',
          '17-Jul', '17-Aug', '17-Sep', '17-Oct', '17-Nov', '17-Dec',
          '18-Jan', '18-Feb', '18-Mar', '18-Apr', '18-May','18-Jun',
          '18-Jul', '18-Aug', '18-Sep', '18-Oct', '18-Nov']

In [ ]:
data.period = data.period.astype('category',categories=months)

In [35]:
data.sort_values(['id','channel' ,'year','period'],inplace=True)

In [36]:
data

,country,channel,brand,model,id,year,period,smart_connect,smart_h._ecosys,operating_ai,airplay,googlecast/home,bluetooth,ethernet,wifi,height_in_mm,width_in_mm,high-res_audio,type_of_docking,multiroom,no.speakerboxes,output_channel,streaming_connection,streaming_technology,usb_connection,wattage_total,sales_units,sales_value_eur,price_eur,sales_value_usd,price_usd
5188,Germany,Consumer Electronic Stores,NOXON,NOVA S,100293423,2016,16-Dec,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,2,238,119,253,126
5006,Germany,Consumer Electronic Stores,NOXON,NOVA S,100293423,2017,17-Jan,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,1,139,139,145,145
4807,Germany,Consumer Electronic Stores,NOXON,NOVA S,100293423,2017,17-Feb,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,1,139,139,150,150
4608,Germany,Consumer Electronic Stores,NOXON,NOVA S,100293423,2017,17-Mar,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,1,100,100,105,105
4408,Germany,Consumer Electronic Stores,NOXON,NOVA S,100293423,2017,17-Apr,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,4,400,100,426,107
3760,Germany,Consumer Electronic Stores,NOXON,NOVA S,100293423,2017,17-Jul,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,52,3294,63,3745,72
3576,Germany,Consumer Electronic Stores,NOXON,NOVA S,100293423,2017,17-Aug,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,48,3854,81,4553,95
3501,Germany,Consumer Electronic Stores,NOXON,NOVA S,100293423,2017,17-Sep,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,1,66,66,79,79
3204,Germany,Consumer Electronic Stores,NOXON,NOVA S,100293423,2017,17-Oct,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,22,1226,57,1440,67
2993,Germany,Consumer Electronic Stores,NOXON,NOVA S,100293423,2017,17-Nov,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,24,1395,59,1625,69


In [37]:
 dfBasedOnID = pd.DataFrame()
# MasterUni = MasterUni.append(MasteruniversityMajorIdDf , ignore_index=True)

uniqueIds = data.id.unique()

In [38]:
uniqueIds

array(['100293423', '100671708', '101024462', '101024494', '101240478',
       '102215785', '102345694', '102402338', '104594624', '104664708',
       '104980264', '105107104', '105112279', '105227304', '105262570',
       '105262595', '105528291', '105568370', '105735826', '105735857',
       '105737242', '105817564', '105873415', '106041850', '106067549',
       '106088160', '106150316', '106150685', '106202226', '106272722',
       '106327636', '106556928', '106557125', '106563503', '106898652',
       '107339258', '107509690', '107596649', '108309426', '108318769',
       '108772475', '108784754', '109229607', '109409193', '109503483',
       '109503603', '109589437', '109591946', '109713835', '110606111',
       '110606283', '110750097', '111075399', '111075560', '111603511',
       '111607890', '112341287', '112576020', '112624845', '112625157',
       '112737335', '112787038', '113358762', '114883726', '115118653',
       '115146739', '115420520', '115522319', '115555873', '1155

In [39]:
data.columns

Index(['country', 'channel', 'brand', 'model', 'id', 'year', 'period',
       'smart_connect', 'smart_h._ecosys', 'operating_ai', 'airplay',
       'googlecast/home', 'bluetooth', 'ethernet', 'wifi', 'height_in_mm',
       'width_in_mm', 'high-res_audio', 'type_of_docking', 'multiroom',
       'no.speakerboxes', 'output_channel', 'streaming_connection',
       'streaming_technology', 'usb_connection', 'wattage_total',
       'sales_units', 'sales_value_eur', 'price_eur', 'sales_value_usd',
       'price_usd'],
      dtype='object')

In [40]:
columnsWithSantization = ['sales_units', 'sales_value_eur', 'price_eur', 'sales_value_usd',
       'price_usd']

In [41]:
for col in columnsWithSantization:
    data[col] = data[col].str.replace(',', '').str.lower()

In [42]:
 dfBasedOnID = pd.DataFrame()
# MasterUni = MasterUni.append(MasteruniversityMajorIdDf , ignore_index=True)

In [43]:
    collectedID = pd.DataFrame(columns = ['country', 'channel', 'brand', 'model', 'id', 'year', 'period',
       'smart_connect', 'smart_h._ecosys', 'operating_ai', 'airplay',
       'googlecast/home', 'bluetooth', 'ethernet', 'wifi', 'height_in_mm',
       'width_in_mm', 'high-res_audio', 'type_of_docking', 'multiroom',
       'no.speakerboxes', 'output_channel', 'streaming_connection',
       'streaming_technology', 'usb_connection', 'wattage_total',
       'sales_units', 'sales_value_eur', 'price_eur', 'sales_value_usd',
       'price_usd'])

In [44]:
for uuid in uniqueIds:
    allDataBasedOnThisID = data[data.id==uuid]
    total_sales_units = 0
    total_sales_value_eur = 0
    total_price_eur = 0 
    total_sales_value_usd = 0
    total_price_usd = 0
    for i, row in allDataBasedOnThisID.iterrows():
        total_sales_units += (int)(row.sales_units)
        total_sales_value_eur += (int)(row.sales_value_eur)
        total_price_eur += (int)(row.price_eur)
        total_sales_value_usd += (int)(row.sales_value_usd)
        total_price_usd += (int)(row.price_usd)
    firstRow = allDataBasedOnThisID.iloc[0]
    firstRow.sales_units = total_sales_units
    firstRow.sales_value_eur = total_sales_value_eur
    firstRow.price_eur = total_price_eur
    firstRow.sales_value_usd = total_sales_value_usd
    firstRow.price_usd = total_price_usd
#     print(firstRow)
#     print(type(firstRow))
    collectedID = collectedID.append(firstRow, ignore_index=True)

/Users/mike/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [47]:
# for i, row in data.iterrows():
#      print(row.id)

In [45]:
collectedID

,country,channel,brand,model,id,year,period,smart_connect,smart_h._ecosys,operating_ai,airplay,googlecast/home,bluetooth,ethernet,wifi,height_in_mm,width_in_mm,high-res_audio,type_of_docking,multiroom,no.speakerboxes,output_channel,streaming_connection,streaming_technology,usb_connection,wattage_total,sales_units,sales_value_eur,price_eur,sales_value_usd,price_usd
0,Germany,Consumer Electronic Stores,NOXON,NOVA S,100293423,2016,16-Dec,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,140,249.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,BT+UPNP,NO USB,unknown,932,61058,3553,72294,4054
1,Germany,Consumer Electronic Stores,PANASONIC,SC-ALL2,100671708,2016,16-Dec,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,NO GOOGLECAST,WITH BLUETOOTH,ETHERNET,WIFI BUILT-IN,174,110.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,n.a.,2,Bluetooth+Ethernet+Wifi,Proprietary +BT,NO USB,40,6718,1019991,12957,1142479,14852
2,Germany,Consumer Electronic Stores,LG,MUSIC FLOW H5 NA9540,101024462,2016,16-Dec,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,ETHERNET,WIFI BUILT-IN,207,340.0,HIGH-RES AUDIO,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Ethernet+Wifi,BT+UPNP,NO USB,40,251,35680,4678,39099,5302
3,Germany,Consumer Electronic Stores,LG,MUSIC FLOW H7 NA9740,101024494,2016,16-Dec,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,ETHERNET,WIFI BUILT-IN,232,370.0,HIGH-RES AUDIO,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Ethernet+Wifi,BT+UPNP,NO USB,70,226,46180,6148,51765,7017
4,Germany,Consumer Electronic Stores,LG,MUSIC FLOW H3 NA9340,101240478,2016,16-Dec,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,n.a.,WITH BLUETOOTH,ETHERNET,WIFI BUILT-IN,175,125.0,HIGH-RES AUDIO,BLUETOOTH ONLY,MULTIROOM,1,0,Bluetooth+Ethernet+Wifi,Bluetooth,USB TYPE-A/B,30,272,35337,3138,38418,3536
5,Germany,Consumer Electronic Stores,SONOS,2 PLAY:1S,102215785,2016,16-Dec,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,GOOGLECAST/HOME,W/O BLUETOOTH,ETHERNET,WIFI BUILT-IN,161.45,119.7,NO HIGH-RES AUD,NO DOCKING/STRE,MULTIROOM,1,2,Wifi+Ethernet,n.a.,NO USB,110,1588,608923,18065,687083,20752
6,Germany,Consumer Electronic Stores,DENON,HEOS 1,102345694,2016,16-Dec,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,NO GOOGLECAST,OPTIONAL,NO ETHERNET,WIFI BUILT-IN,189,129.0,HIGH-RES AUDIO,NO DOCKING/STRE,MULTIROOM,1,2,Wifi,UPNP,USB TYPE-A/B,unknown,3486,441237,6932,493138,7935
7,Germany,Consumer Electronic Stores,SAMSUNG,WAM 7500/WAM 7501 / SPEAKER R7,102402338,2016,16-Dec,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,NO GOOGLECAST,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,422,181.0,HIGH-RES AUDIO,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,Bluetooth,USB TYPE-A/B,80,8448,2386103,26750,2744632,30498
8,Germany,Consumer Electronic Stores,POLK AUDIO,OMNI S2,104594624,2016,16-Dec,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,NO GOOGLECAST,W/O BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,100,230.0,NO HIGH-RES AUD,NO DOCKING/STRE,MULTIROOM,1,2,Wifi,n.a.,NO USB,unknown,53,9182,5535,10610,6351
9,Germany,Consumer Electronic Stores,POLK AUDIO,OMNI S2R,104664708,2017,17-Jan,SMART APP CTRL.,n.a.,n.a.,NO AIRPLAY,NO GOOGLECAST,WITH BLUETOOTH,NO ETHERNET,WIFI BUILT-IN,76,219.0,NO HIGH-RES AUD,BLUETOOTH ONLY,MULTIROOM,1,2,Bluetooth+Wifi,n.a.,NO USB,unknown,21,4305,2440,4959,2795


In [46]:
collectedID.describe()

,width_in_mm
count,268.000000
mean,249.466045
std,157.371441
min,51.900000
25%,135.000000
50%,215.000000
75%,333.250000
max,1000.000000
